# 🌐 Connect to google drive

In [ ]:
# Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# H5 ➡️ ONNX

In [ ]:
!pip install tensorflow
!pip install tf2onnx
!pip install onnx

In [ ]:
# Select h5 model path
h5_model_path = '/content/drive/MyDrive/saved_models/mobileNetAug-CIFAR10-95.h5'
# Select where to save the onnx model
onnx_model_save_path = '/content/drive/MyDrive/saved_models/test.onnx'

In [ ]:
import tensorflow as tf
import tf2onnx
import onnx
from tensorflow.keras.models import load_model

model = load_model(h5_model_path)
# Create a concrete function from the model
input_signature = [tf.TensorSpec(shape=[None] + list(model.input_shape[1:]), dtype=tf.float32, name="input")]
# Convert the model to ONNX Format
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature=input_signature)
onnx.save(onnx_model, onnx_model_save_path)

# ONNX ➡️ Protobuff / tflite


In [ ]:
!pip install onnx2tf
!pip install onnx-graphsurgeon
!pip install sng4onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.5/147.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 79.0 MB/s eta 0:00:00


In [ ]:
%cd /content/drive/MyDrive/saved_models/onnx_models

/content/drive/MyDrive/saved_models/onnx_models


In [ ]:
!onnx2tf --version

E0000 00:00:1741376666.829790    3050 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741376666.853809    3050 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
1.26.8


In [ ]:
!onnx2tf -h

E0000 00:00:1740245668.109332   10310 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740245668.131455   10310 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
usage: onnx2tf [-h] (-i INPUT_ONNX_FILE_PATH | -V) [-o OUTPUT_FOLDER_PATH] [-osd] [-oh5] [-okv3]
               [-otfv1pb] [-ow] [-coion] [-odrqt] [-oiqt] [-qt {per-channel,per-tensor}]
               [-cind CUSTOM_INPUT_OP_NAME_NP_DATA_PATH [CUSTOM_INPUT_OP_NAME_NP_DATA_PATH ...]]
               [-iqd {int8,uint8,float32}] [-oqd {int8,uint8,float32}] [-nuo] [-nuonag]
               [-b BATCH_SIZE] [-ois OVERWRITE_INPUT_SHAPE [OVERWRITE_INPUT_SHAPE ...]] [-nlt]
               [-onwdt] [-snms {v4,v5}]
               [-k KEEP_NCW_OR_NCHW_OR_NCDHW_INPUT_NAMES [KEEP_NCW_OR_NCHW_OR_NCDHW_INPUT_NAMES ...]]
               [-kt KEEP_NWC_OR_NHWC_OR

In [ ]:
import tensorflow as tf
import onnx
from tensorflow.keras.models import load_model
onnx_model_path = '/content/drive/MyDrive/saved_models/onnx_models/mobile_pruned_finetuned.onnx'
model = onnx.load(onnx_model_path)

# Check input details
for input_tensor in model.graph.input:
    print(f"Input Name: {input_tensor.name}")
    shape = [dim.dim_value for dim in input_tensor.type.tensor_type.shape.dim]
    print(f"Input Shape: {shape}")
    print(f"Data Type: {input_tensor.type.tensor_type.elem_type}")

Input Name: input
Input Shape: [1, 3, 224, 224]
Data Type: 1


In [ ]:
!onnx2tf -i mobile_pruned_finetuned.onnx -o mobile_pruned_finetuned -k KEEP_NCW_OR_NCHW_OR_NCDHW_INPUT_NAMES input

E0000 00:00:1741376760.568247    3448 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741376760.589071    3448 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered

Model optimizing started ============================================================
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/onnx2tf/onnx2tf.py", line 652, in convert
    result = subprocess.check_output(
             ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 466, in check_output
    return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Fi

# Testing the converted model

## Loading dataset

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,RandomFlip, RandomRotation, RandomZoom
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time
import random
import matplotlib.pyplot as plt
import os
import pandas as pd
# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(f"Original Train set: {x_train.shape}")
print(f"Original Test set: {x_test.shape}")
# Reserve 5000 samples from the training set for validation
x_train, x_dev = x_train[:45000], x_train[45000:]
y_train, y_dev = y_train[:45000], y_train[45000:]

print(f"Train set: {x_train.shape}")
print(f"Dev set: {x_dev.shape}")
print(f"Test set: {x_test.shape}")
# Define batch size
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE  # Optimizes performance
# Define batch size
BATCH_SIZE = 64
AUTOTUNE = tf.data.AUTOTUNE  # Optimizes performance
# Define data augmentation layers
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),  # Random horizontal flipping
    RandomRotation(0.1),       # Random rotation (10% of 360 degrees)
    RandomZoom(0.1)            # Random zoom
])

# Function to resize and normalize images
def preprocess(image, label, augment=False):
    image = tf.image.resize(image, (224, 224))  # Resize dynamically

    if augment:
        image = data_augmentation(image)  # Apply augmentations
    image = preprocess_input(image)  # Normalize for MobileNet
    return image, label

# Convert datasets to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dev_dataset = tf.data.Dataset.from_tensor_slices((x_dev, y_dev))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Apply preprocessing and batching
train_dataset = (
    tf.data.Dataset.from_tensor_slices((x_train, y_train))
    .shuffle(buffer_size=10000)  # Shuffle data for randomness
    .map(lambda x, y: preprocess(x, y, augment=True), num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)
dev_dataset = (
    dev_dataset
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    test_dataset
    .map(preprocess, num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

# Print dataset structure
print(train_dataset)
print(dev_dataset)
print(test_dataset)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Original Train set: (50000, 32, 32, 3)
Original Test set: (10000, 32, 32, 3)
Train set: (45000, 32, 32, 3)
Dev set: (5000, 32, 32, 3)
Test set: (10000, 32, 32, 3)
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.uint8, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.uint8, name=None))>
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.uint8, name=None))>


In [ ]:
# Take only the first 1000 images from the test dataset
num_images = 1000
inference_testset = test_dataset.take(num_images // BATCH_SIZE)  # Take enough batches
# print the length of the inference testset
test_img_count = 0
for images, _ in inference_testset:
    test_img_count += images.shape[0]  # Add the batch size each time

print(f"Subset contains {test_img_count} images")

Subset contains 960 images


### converting protobuff to tflite

In [ ]:
original_model_path = '/content/drive/MyDrive/saved_models/tf_model'

# Load the model
model = tf.saved_model.load(original_model_path)
concrete_func = model.signatures["serving_default"]  # Explicitly get the function
# Convert using the concrete function
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

# Save the TFLite model
tflite_original_model_path = "/content/drive/My Drive/saved_models/test.tflite"
with open(tflite_original_model_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved at: {tflite_original_model_path}")

TFLite model saved at: /content/drive/My Drive/saved_models/test.tflite


### Running inference

In [ ]:
# Function to run inference on TFLite model with CIFAR-10 dataset
def evaluate_tflite_model(interpreter, dataset):
    correct = 0
    total = 0

    # Get input and output tensor details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Get quantization scale and zero point for input and output tensors
    input_scale, input_zero_point = input_details[0]['quantization']
    output_scale, output_zero_point = output_details[0]['quantization']
    start_time = time.time()

    for images, labels in dataset:
        for i in range(len(images)):
            # Prepare input image
            input_data = np.expand_dims(images[i].numpy(), axis=0)

            # Ensure proper INT8 conversion if the model is quantized
            if input_details[0]['dtype'] == np.int8:
                input_data = ((input_data / input_scale) + input_zero_point).astype(np.int8)
            else:
                input_data = input_data.astype(np.float32)

            # Set the input tensor
            interpreter.set_tensor(input_details[0]['index'], input_data)

            # Run inference
            interpreter.invoke()

            # Get the output tensor
            output_data = interpreter.get_tensor(output_details[0]['index'])

            # If the model is quantized (INT8), dequantize output
            if output_details[0]['dtype'] == np.int8:
                output_data = (output_data.astype(np.float32) - output_zero_point) * output_scale

            # Get the predicted class (highest probability)
            predicted_label = np.argmax(output_data, axis=1)[0]
            # Compare with ground truth
            if predicted_label == labels[i].numpy():
                correct += 1
            total += 1
    end_time  = time.time()
    execution_time = end_time - start_time
    accuracy = (correct / total) * 100
    return accuracy, execution_time


In [ ]:
tflite_original_model_path = '/content/drive/MyDrive/saved_models/tf_model/test_float32.tflite'

In [ ]:
# Load the model
interpreter = tf.lite.Interpreter(model_path=tflite_original_model_path)
interpreter.allocate_tensors()
# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input details:", input_details)

Input details: [{'name': 'input', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [ ]:
# Run evaluation on TFLite model
original_accuracy, original_execution_time = evaluate_tflite_model(interpreter, inference_testset)
print(f"TFLite Model Accuracy: {original_accuracy:.2f} %")
print(f"TFLite Inference Time for {test_img_count} images: {original_execution_time:.2f} s")

TFLite Model Accuracy: 94.48 %
TFLite Inference Time for 960 images: 16.80 s
